# Global registration with RANSAC
We are going to use [open3d](http://www.open3d.org/) to handle point clouds and generation of point clouds
We are importing the packages and defining a function which helps us drawing the point clouds.

In [1]:
import open3d as o3d

import numpy as np
import copy

# helper function for drawing
# If you want it to be more clear set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

/Users/madsyar/Desktop/1. semester/Perception for autonomous systems/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


We need to read in our pointclouds. For that we use the `io` module of the
open3d package (`o3d`). The following cell will open a window with a
visualization of both point clouds `source` and `target`.

Also, this page [Visualization - Open3D](http://open3d.org/html/tutorial/Basic/visualization.html)
contains some useful examples and instructions on how to use the viewer.

In [2]:
source = o3d.io.read_point_cloud("ICP/r1.pcd")
target = o3d.io.read_point_cloud("ICP/r2.pcd")
target2 = o3d.io.read_point_cloud("ICP/r3.pcd")

# Used for downsampling.
voxel_size = 0.05

# Show models side by side
#draw_registrations(source, target)

### Finding features in pointclouds
When working on point clouds it can be beneficial to work on a downsampled version of the point cloud,
as it decreases the need of computation.

You can use [`pointcloud.voxel_down_sample()`](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html#open3d.geometry.PointCloud.voxel_down_sample) where `pointcloud` is the name of your point cloud object. In our case, that would be `source` and `target`.

We also need to estimate the normals of the point cloud points using [`pointcloud.estimate_normals()`](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html#open3d.geometry.PointCloud.voxel_down_sample)

**Task:** Find FPFH features or correspondances of the downsampled point clouds.
[`o3d.pipelines.registration.compute_fpfh_feature()`](http://www.open3d.org/docs/latest/python_api/open3d.pipelines.registration.compute_fpfh_feature.html)


In [3]:
####
# Downsample and find features here
####

# Code
source_down = source.voxel_down_sample(voxel_size)
target_down = target.voxel_down_sample(voxel_size)
target2_down = target.voxel_down_sample(voxel_size)

source_down.estimate_normals()
target_down.estimate_normals()
target2_down.estimate_normals()

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))

target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target_down,o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))

target2_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target2_down,o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))


### RANSAC 
We will now attempt to use RANSAC to do a global registration of the two point clouds.

By using the function [`o3d.pipelines.registration.registration_ransac_based_on_feature_matching`](http://www.open3d.org/docs/latest/python_api/open3d.pipelines.registration.registration_ransac_based_on_feature_matching.html) from open3d, do the following:


Try to find the transformation from `r1.pcd` (`source`) to `r2.pcd` (`target`).
Attempt with point-to-point and point-to-plane
```Python
point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()
```

When using RANSAC, focus on the arguments below. The rest are optional parameters.
```Python
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point)
```

In [4]:
####
# Call RANSAC here
####
point_to_point = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane = o3d.pipelines.registration.TransformationEstimationPointToPlane()

distance_threshold = voxel_size*1.5

# ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
#     source_down, target_down, source_fpfh, target_fpfh,
#     mutual_filter=False,  # Set to True if you want to enforce mutual correspondence filtering
#     max_correspondence_distance=distance_threshold,
#     estimation_method=point_to_point,
#     ransac_n=4,  # Number of points to sample for RANSAC
#     checkers=[
#         o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
#         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
#     ],
#     criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
# )

# ransac_result2 = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
#     source_down, target2_down, source_fpfh, target2_fpfh,
#     mutual_filter=False,  # Set to True if you want to enforce mutual correspondence filtering
#     max_correspondence_distance=distance_threshold,
#     estimation_method=point_to_point,
#     ransac_n=4,  # Number of points to sample for RANSAC
#     checkers=[
#         o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
#         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
#     ],
#     criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
# )

#draw_registrations(source, target2, ransac_result2.transformation, True)

## Exercises
### A)
Can you get a decent transformation from r1 to r3? (check the ICP folder)

1. Kinda. But it is a bit off
### B)
With the following checkers, can you get better results from RANSAC? Try tweaking the parameters of them. Can you make point-to-plane work? Do not spend too much time on this, if you can't manage, skip it. (I was not able to get a good fit.)

You can also try tweaking the `voxel_size`

```Python
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0,c1,c2]

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point,
    checkers = checker_list)
```


In [5]:
corr_length = 0.9
voxel_size = 0.1
distance_threshold = voxel_size * 1.5

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0,c1,c2]

ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source, target, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point,
    checkers = checker_list)

draw_registrations(source, target, ransac_result.transformation, True)

: 

Nope, I could not get a better result than the RANSAC